### For the second week, the final deliverables of the project will be:
1. A link to your Notebook on your Github repository, showing your code. (15 marks)
2. A full report consisting of all of the following components (15 marks):
* Introduction where you discuss the business problem and who would be interested in this project.
* Data where you describe the data that will be used to solve the problem and the source of the data.
* Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
* Results section where you discuss the results.
* Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
* Conclusion section where you conclude the report.
3. Your choice of a presentation or blogpost. (10 marks)

In [70]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [77]:
CLIENT_ID = 'R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K' # your Foursquare ID
CLIENT_SECRET = '4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K
CLIENT_SECRET:4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW


In [91]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "Restaurant",
    "near": "Chiang Mai",
    "radius": 20000,
    "limit": 200}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [92]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [93]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Chiang Mai', 'Chiang Mai')

In [94]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 18.81595128750102, 'lng': 99.01119209195646},
  'sw': {'lat': 18.76652525718335, 'lng': 98.9520826989491}},
 203)

In [95]:
d["geocode"]

{'what': '',
 'where': 'chiang mai',
 'center': {'lat': 18.79038, 'lng': 98.98468},
 'displayString': 'Chiang Mai, Thailand',
 'cc': 'TH',
 'geometry': {'bounds': {'ne': {'lat': 19.087014307381935,
    'lng': 99.16659946870921},
   'sw': {'lat': 18.642158022985413, 'lng': 98.86197488261813}}},
 'slug': 'chiang-mai-thailand',
 'longId': '72057594039081607'}

In [96]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [97]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [98]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4c2ae2aa57a9c9b6e02ff567',
  'name': 'Wat Phra Singh Waramahavihan (วัดพระสิงห์วรมหาวิหาร)',
  'location': {'address': 'Samlan Rd',
   'crossStreet': 'Ratchadamnoen Rd',
   'lat': 18.78857860553444,
   'lng': 98.98205251033949,
   'labeledLatLngs': [{'label': 'display',
     'lat': 18.78857860553444,
     'lng': 98.98205251033949}],
   'postalCode': '50000',
   'cc': 'TH',
   'city': 'เมืองเชียงใหม่',
   'state': 'เชียงใหม่',
   'country': 'ประเทศไทย',
   'formattedAddress': ['Samlan Rd (Ratchadamnoen Rd)',
    'เมืองเชียงใหม่',
    'เชียงใหม่ 50000',
    'ประเทศไทย']},
  'categories': [{'id': '52e81612bcbc57f1066b7a3e',
    'name': 'Buddhist Temple',
    'pluralName': 'Buddhist Temples',
    'shortName': 'Buddhist Temple',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/religious_',
     'suffix': '.pn

In [99]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i Restaurant" % len(df))
df.head()

{'address': 'Samlan Rd', 'crossStreet': 'Ratchadamnoen Rd', 'lat': 18.78857860553444, 'lng': 98.98205251033949, 'labeledLatLngs': [{'label': 'display', 'lat': 18.78857860553444, 'lng': 98.98205251033949}], 'postalCode': '50000', 'cc': 'TH', 'city': 'เมืองเชียงใหม่', 'state': 'เชียงใหม่', 'country': 'ประเทศไทย', 'formattedAddress': ['Samlan Rd (Ratchadamnoen Rd)', 'เมืองเชียงใหม่', 'เชียงใหม่ 50000', 'ประเทศไทย']}
{'address': '91/1-2 Sri Poom Rd.', 'crossStreet': 'inside the moat just east of Chang Puak Gate', 'lat': 18.795339145276785, 'lng': 98.98704445728309, 'labeledLatLngs': [{'label': 'display', 'lat': 18.795339145276785, 'lng': 98.98704445728309}], 'postalCode': '50200', 'cc': 'TH', 'city': 'เมืองเชียงใหม่', 'state': 'เชียงใหม่', 'country': 'ประเทศไทย', 'formattedAddress': ['91/1-2 Sri Poom Rd. (inside the moat just east of Chang Puak Gate)', 'เมืองเชียงใหม่', 'เชียงใหม่ 50200', 'ประเทศไทย']}
{'address': '175/6 Ratchadamnoen Rd', 'lat': 18.788487621742455, 'lng': 98.9831164274197

AttributeError: 'NoneType' object has no attribute 'items'

                        uid  \
0  4c2ae2aa57a9c9b6e02ff567   
1  4b67fa22f964a520b3622be3   
2  4b63147df964a52087612ae3   
3  51b98562498eb59c591d4139   
4  4c414ed4ce54e21e73fd0a1a   

                                                name        shortname address  \
0  Wat Phra Singh Waramahavihan (วัดพระสิงห์วรมหา...  Buddhist Temple           
1                              North Gate Jazz Co-Op        Jazz Club           
2                                  Rachamankha Hotel            Hotel           
3                        Akha Ama Coffee La Fattoria      Coffee Shop           
4                   Khao Soi Mae Sai (ข้าวซอยแม่สาย)          Noodles           

  postalcode        lat        lng  
0      50000  18.788579  98.982053  
1      50200  18.795339  98.987044  
2      50200  18.787145  98.980127  
3      50200  18.788393  98.983314  
4      50300  18.799582  98.975257  

In [100]:
ChiangMai_center = d["geocode"]["center"]
ChiangMai_center

{'lat': 18.79038, 'lng': 98.98468}

In [101]:
from folium import plugins


map_ChiangMai = folium.Map(location=[18.79038,98.98468], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_ChiangMai)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_ChiangMai.add_child(plugins.HeatMap(hm_data))

map_ChiangMai

In [102]:
lat = 18.79038
lng = 98.98468
map_DaNang = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_DaNang)
map_DaNang